In [ ]:
!pip install concrete-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.4/200.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.6/164.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.6/671.6 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import torch
# import tensorflow as tf
import pandas as pd
from google.colab import files


uploaded = files.upload()

Saving heart.csv to heart.csv


In [ ]:
dataset = pd.read_csv('heart.csv')
target_vector = dataset['target'].values
target_vector
design = dataset.drop(['target'], 'columns')

<ipython-input-2-478162f23150>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  design = dataset.drop(['target'], 'columns')


In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from concrete.ml.sklearn import LogisticRegression
import time

# Lets create a synthetic data-set
x, y = make_classification(n_samples=100, class_sep=2, n_features=30, random_state=42)

# Split the data-set into a train and test set
X_train, X_test, y_train, y_test = train_test_split(
    design, target_vector, test_size=0.2, random_state=42
)

# Now we train in the clear and quantize the weights
model = LogisticRegression(n_bits=16)
model.fit(X_train, y_train)

# We can simulate the predictions in the clear
clear_start_time = time.time()
y_pred_clear = model.predict(X_test)
print(f"Clear: {time.time() - clear_start_time}")

# We then compile on a representative set
model.compile(X_train)

# Finally we run the inference on encrypted inputs
en_start_time = time.time()
y_pred_fhe = model.predict(X_test, fhe="execute")
print(f"En: {time.time() - en_start_time}")

# Simulate
sim_start_time = time.time()
y_pred_fhe_sim = model.predict(X_test, fhe="simulate") # Only three modes: disable, execute, simulate.
print(f"En: {time.time() - sim_start_time}")

print(f"In clear  : {y_pred_clear}")
print(f"In FHE    : {y_pred_fhe}")
print(f"Similarity: {(y_pred_fhe == y_pred_clear).mean():.1%}")
print(y_pred_fhe_sim)
print(f"Similarity: {(y_pred_fhe == y_pred_fhe_sim).mean():.1%}")

Clear: 0.0018756389617919922
En: 0.28847765922546387
En: 0.06071209907531738
In clear  : [1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1
 0 1 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1
 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1
 1 1 0 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1
 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0]
In FHE    : [1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1
 0 1 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1
 1 1 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1
 1 1 0 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 1
 1 0 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0]
Similarity: 100.0%
[1 1 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 1 1 0 1 0 1

In [ ]:
# Training on clear data
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from concrete.ml.sklearn import LogisticRegression
import numpy

# Create a synthetic data-set for a classification problem
x, y = make_classification(n_samples=100, class_sep=2, n_features=3, n_informative=3, n_redundant=0, random_state=42)

# Split the data-set into a train and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Instantiate and train the model
model = LogisticRegression()
model.fit(x_train,y_train)

# Simulate the predictions in the clear (optional)
y_pred_clear = model.predict(x_test)

# Compile the model on a representative set
fhe_circuit = model.compile(x_train)

# Predict in FHE
y_pred_fhe = model.predict(x_test, fhe="execute")

# Generate the keys (set force to True in order to generate new keys at each execution)
fhe_circuit.keygen(force=True)

y_pred_fhe_step = []

for f_input in x_test:
    # Quantize an input (float)
    q_input = model.quantize_input([f_input])

    # Encrypt the input
    q_input_enc = fhe_circuit.encrypt(q_input)

    # Execute the linear product in FHE
    q_y_enc = fhe_circuit.run(q_input_enc)

    # Decrypt the result (integer)
    q_y = fhe_circuit.decrypt(q_y_enc)

    # De-quantize the result
    y = model.dequantize_output(q_y)

    # Apply either the sigmoid if it is a binary classification task, which is the case in this
    # example, or a softmax function in order to get the probabilities (in the clear)
    y_proba = model.post_processing(y)

    # Since this model does classification, apply the argmax to get the class predictions (in the clear)
    # Note that regression models won't need the following line
    y_class = numpy.argmax(y_proba, axis=1)

    y_pred_fhe_step += list(y_class)

y_pred_fhe_step = numpy.array(y_pred_fhe_step)

print("Predictions in clear:", y_pred_clear)
print("Predictions in FHE  :", y_pred_fhe_step)
print(f"Similarity: {int((y_pred_fhe_step == y_pred_clear).mean()*100)}%")
print(y_test)

Predictions in clear: [1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 0]
Predictions in FHE  : [1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 0]
Similarity: 100%
[1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 0]


In [ ]:
# Training on encrypted data
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from concrete.ml.sklearn import LogisticRegression
import numpy

x, y = make_classification(n_samples=100, class_sep=2, n_features=3, n_informative=3, n_redundant=0, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


model = LogisticRegression()
model.fit(x_train,y_train)
fhe_circuit = model.compile(x_train)


# Simulate the predictions in the clear (optional)
y_pred_clear = model.predict(x_test)

# Predict in FHE
y_pred_fhe = model.predict(x_test, fhe="execute")

y_pred_fhe_step = []

for f_input in x_test:
    print(f_input)
    q_input = model.quantize_input([f_input]) # Quantize an input (float)
    print(q_input)
    q_input_enc = fhe_circuit.encrypt(q_input) # Encrypt the input
    print(q_input_enc)
    q_y_enc = fhe_circuit.run(q_input_enc) # Execute the linear product in FHE
    print(q_y_enc)
    q_y = fhe_circuit.decrypt(q_y_enc) # Decrypt the result (integer)
    print(q_y)
    y = model.dequantize_output(q_y) # De-quantize the result
    print(y)

    # Apply either the sigmoid if it is a binary classification task, which is the case in this
    # example, or a softmax function in order to get the probabilities (in the clear)
    y_proba = model.post_processing(y)
    print(y_proba)

    y_class = numpy.argmax(y_proba, axis=1) # For classification
    print(y_class)

    y_pred_fhe_step += list(y_class)

y_pred_fhe_step = numpy.array(y_pred_fhe_step)

print("Predictions in clear:", y_pred_clear)
print("Predictions in FHE  :", y_pred_fhe_step)
print(f"Similarity: {int((y_pred_fhe_step == y_pred_clear).mean()*100)}%")
print(y_pred_fhe_step)
print(y_test)

[0.82335334 0.2187486  2.9436449 ]
[[33 17 89]]
[[34848]]
[[4.99436048]]
[[0.00673045 0.99326955]]
[1]
[1.81376537 0.64854061 2.21094914]
[[59 28 70]]
[[33855]]
[[4.77270141]]
[[0.00838657 0.99161343]]
[1]
[ 1.8755859  -2.59145991  3.21543669]
[[ 61 -58  96]]
[[36565]]
[[5.377632]]
[[0.00459751 0.99540249]]
[1]
[ 3.3986169  -2.00304259  1.2286702 ]
[[101 -42  44]]
[[27549]]
[[3.36506589]]
[[0.03340526 0.96659474]]
[1]
[1.6730551  1.01925929 2.32952878]
[[55 38 73]]
[[34863]]
[[4.9977088]]
[[0.0067081 0.9932919]]
[1]
[-0.10612355  3.78052765  3.39549236]
[[  8 111 101]]
[[41184]]
[[6.40869265]]
[[0.00164447 0.99835553]]
[1]
[-0.56209771  4.53580658  3.68984314]
[[ -4 127 109]]
[[42888]]
[[6.78906229]]
[[0.00112476 0.99887524]]
[1]
[-3.39453293 -5.64562924 -1.6231791 ]
[[ -79 -128  -32]]
[[-33933]]
[[-10.35904562]]
[[9.99968296e-01 3.17036948e-05]]
[0]
[3.26201789 1.63449524 1.14823048]
[[98 54 41]]
[[32877]]
[[4.55439067]]
[[0.01041137 0.98958863]]
[1]
[ 0.9449532  -1.07689964  2.014236